In [1]:
# default_exp core.store
# export
from ascrum.core.config import Config
from pathlib import Path
import yaml
import os
import time
import getpass

In [2]:
# export
class Store:
    def __init__(self, path):
        self.path = path.resolve()
        self.name = path.stem

    def exists(self):
        return self.path.exists()

    def __getitem__(self, story_id):
        "Get the content of a story"
        p = self.get_path(story_id)
        if not p.exists():
            return None
        with p.open() as f:
            return yaml.load(f, Loader=yaml.FullLoader)

    def __setitem__(self, story_id, story):
        p = self.get_path(story_id)
        with p.open('w') as f:
            yaml.dump(story, f)
        return p.stem

    def __delitem__(self, story_id):
        p = self.get_path(story_id)
        if p.exists():
            p.unlink()
        else:
            raise KeyError(story_id)

    def get_path(self, story_id):
        return self.path/f'{story_id}.yaml'

    def touch(self, story_id):
        p = self.get_path(story_id)
        if p.exists():
            p.touch(exist_ok=True)
            return True 
        else:
            return False

    def keys(self):
        return [f.stem for f in sorted(self.path.glob('*.yaml'), key=os.path.getmtime, reverse=True)]